In [1]:
import configparser
import warnings
from pymongo import MongoClient

warnings.filterwarnings("ignore", category=DeprecationWarning)

config = configparser.ConfigParser()
config.read('../config.ini')
host = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

client = MongoClient(host, int(port))
db = client['Media-cloud']

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

file_names = [
    'domestic-violence-stories (8640).csv',  
    'family-violence-stories (4014).csv'
]

in_path = '../input/'
out_path = '../output/'

In [20]:
def get_text(url):
    if '/?nk=' in url:
        return ''
        
    text = ''
    try:
        html = requests.get(url).text
        soup = BeautifulSoup(html)

        for p in soup.find_all('p'):
            if p.parent.get('class') == ['section']:
                continue
            if p.get('class') == ['_2g0-m']:
                continue
            sentence = p.get_text().strip()
            if (not sentence.startswith('A NOTE ABOUT RELEVANT ADVERTISING') and
                not sentence.startswith('If you have inside knowledge') and
                not sentence.startswith('ABC teams share the story') and
                not sentence.startswith('Read about our editorial') and
                not sentence.startswith('Sign up to receive')
               ) and (
                sentence.endswith('.') or 
                sentence.endswith("'") or
                sentence.endswith('"') or
                sentence.endswith("?") or
                sentence.endswith("!") or
                sentence.endswith(";") or
                sentence.endswith(",") or
                sentence.endswith(")") or
                sentence.endswith(":")
            ):
                text += sentence + ' '
                
        return text.strip()
    except:
        return text.strip()

In [24]:
import json

df = pd.read_csv(in_path+file_names[0])
df['text'] = ''
for i, row in df.iterrows():
    url = row['url']
    df.at[i, 'text'] = get_text(url)

col = db['domestic-violence']
records = json.loads(df.to_json(orient='records'))
for record in records:
    col.insert_one(record)
    
df = pd.read_csv(in_path+file_names[1])
df['text'] = ''
for i, row in df.iterrows():
    url = row['url']
    df.at[i, 'text'] = get_text(url)

col = db['family-violence']
records = json.loads(df.to_json(orient='records'))
for record in records:
    col.insert_one(record)